In [37]:
import numpy as np
from math import floor, ceil
from random import randint, choice
from scipy.stats import norm
import matplotlib.pyplot as plt
import pandas as pd
import os
import re
from tqdm import tqdm
from tqdm.autonotebook import tqdm


### Functions ###

In [38]:
def stacking_uw_plot(outfile_path: str, data: np.ndarray, step_wf_to_plot, time, 
                    remove_starting_noise, highlight_start, highlight_end, 
                    time_ticks_waveforms, ylim_plot, xlim_plot, formato):

    outfile_name = os.path.basename(outfile_path)
    f = plt.figure(figsize = (13,4))
    plt.title("Stacked Waveforms of " + outfile_name, fontsize = 12)
    for wf in range(0,len(data), step_wf_to_plot):
        plt.plot(time[remove_starting_noise:], data[wf][remove_starting_noise:], color = 'black', linewidth = 0.8, alpha = 0.5)
    plt.plot(time[highlight_start:highlight_end], data[wf][highlight_start:highlight_end], color = 'red')
    plt.xlabel('Time [$\mu s$]', fontsize = 12)
    plt.ylabel('Amplitude [.]', fontsize = 12)
    plt.xticks(time_ticks_waveforms)
    plt.ylim(-ylim_plot,ylim_plot)
    plt.xlim(time[0], xlim_plot)
    plt.grid(alpha = 0.1)
    plt.savefig(outfile_path + formato, dpi = 300)
    plt.close()
    #print(outfile_path)
    
def amplitude_map(outfile_path,data, amp_scale, time_info, formato) -> None:
    outfile_name = os.path.basename(outfile_path)
    fig,ax1 = plt.subplots(ncols=2,figsize=(15,8))
    ax1 = plt.subplot()
    ax1.set_title('Amplitude Map of ' + outfile_name, fontsize = 12)
    cmap = plt.get_cmap('seismic')
    im = ax1.imshow(data.T, aspect='auto',origin='lower',interpolation='none',cmap = cmap, vmin = -amp_scale, vmax = amp_scale, extent = [0,data.shape[0]/10, 0, time_info[2]*time_info[3]])
    
    cbar= fig.colorbar(im,pad=0.04)
    cbar.set_label("Relative Amplitude", fontsize = 14)

    ax1.set_xlabel('Time [s]', fontsize = 12)
    ax1.set_ylabel('Wf_time [$\mu s$] ', fontsize = 12)
    ax1.set_xscale

    fig.tight_layout()
    
    plt.savefig(outfile_path + formato, dpi = 300)
    plt.close()
    

def handling_empty_lines(infile, encoding, number_of_samples) -> list[list[str]]:
    data_list = []             
    for line in infile:
        line = line.decode(encoding)
        line = line.split("\t")
        if len(line) < number_of_samples:   #The empty lines are surely shorter than wf lines (determined by the number of samples)
            continue
        data_list.append(line)
    return data_list


## SPECTRAL ANALYSIS

def lowpass_mask(data_length, winlen):
    # winlen       # length of hanning window: the smaller, the lower the cutoff frequency

    # BUILD LOW PASS FILTER
    mask = np.hanning(winlen)
    # mask = np.kaiser(winlen,beta=10)
    mask_start = mask[round(winlen/2):]
    mask_end = mask[:round(winlen/2)]

    lowpass_filter = np.zeros(data_length)
    lowpass_filter[0:len(mask_start)] = mask_start
    lowpass_filter[data_length-len(mask_end):] = mask_end

    return lowpass_filter


def filtered_amp_spectrum_plot(signal_freqs,amp_spectrum, filtered_amp_spectrum, lowpass_filter ):
    print(amp_spectrum.shape)
    # VISUALIZE ONLY REAL PART OF THE SPECTRUM
    spectrum_length = round(len(amp_spectrum)/2)            # needed to make "manually" the real spectrum. I do not trust scipy, sorry
    amp_spectrum = amp_spectrum[:spectrum_length]
    filtered_amp_spectrum = filtered_amp_spectrum[:spectrum_length]
    signal_freqs = signal_freqs[:spectrum_length]
    lowpass_filter = lowpass_filter[:spectrum_length]

    max_freq =signal_freqs[np.argmax(amp_spectrum)]
    
    f = plt.figure(figsize = (13,4))

    plt.semilogy(signal_freqs,amp_spectrum, label = "Amplitude Spectrum")
    plt.semilogy(signal_freqs, lowpass_filter*np.amax(amp_spectrum), label = "Filter Shape")
    plt.semilogy(signal_freqs, filtered_amp_spectrum, label = "Amplitude Spectrum Filtered")
    plt.vlines(max_freq, np.amin(filtered_amp_spectrum),np.amax(filtered_amp_spectrum), "r", 
               "--", label= f"Maximum of the spectrum = {max_freq:.2f} MHz")
    plt.vlines(2, np.amin(filtered_amp_spectrum),np.amax(filtered_amp_spectrum), "r", 
            "-", label= f"Resonce Frequency of the sensor = 2 MHz")
    plt.legend()
    plt.xlim([0,np.amax(signal_freqs)])
    plt.ylabel("Amplitude [.]")
    plt.xlabel("Frequency [MHz]")

    return
    

def phase_spectrum_plot(signal_freqs, phase_spectrum, sampling_rate):
     # VISUALIZE ONLY REAL PART OF THE SPECTRUM
    spectrum_length = round(len(phase_spectrum)/2)            # needed to make "manually" the real spectrum. I do not trust scipy, sorry
    signal_freqs = signal_freqs[:spectrum_length]
    phase_spectrum = phase_spectrum[:spectrum_length]

    f = plt.figure(figsize = (13,4))
    # plt.phase_spectrum(phase_spectrum, 1/sampling_rate)

    plt.plot(signal_freqs,phase_spectrum)

    # plt.xlim([0, 5])

    # Set y-axis ticks and labels from -π to +π
    plt.yticks(np.linspace(-np.pi, np.pi, 5),
            [r'$-\pi$', r'$-\frac{\pi}{2}$', r'$0$', r'$\frac{\pi}{2}$', r'$\pi$'])
    plt.ylabel("Phase [rad]")
    plt.xlabel("Frequency [MHz]")


def signal2noise_separation_lowpass(outfile_path, waveform_data,time_info, freq_cut=2):
    sampling_rate = time_info[3]
    try:
        wave_num, wave_len = waveform_data.shape
    except ValueError:
        wave_num = 1
        wave_len = len(waveform_data)


    amp_spectrum = np.abs(np.fft.fft(waveform_data))
    phase_spectrum = np.angle(np.fft.fft(waveform_data))
    signal_freqs = np.fft.fftfreq(wave_len, sampling_rate)  
    
    # BUILD THE LOWPASS FILTER
    winlen = int((wave_len*freq_cut)/np.amax(signal_freqs))
    lowpass_filter = lowpass_mask(wave_len, winlen)

    # FILTERING THE SPECTRUM
    filtered_amp_spectrum = lowpass_filter * amp_spectrum
    filtered_fourier = filtered_amp_spectrum * np.exp(1j*phase_spectrum)

    if wave_num == 1:
        print(wave_num)
        filtered_amp_spectrum_plot(signal_freqs,amp_spectrum.T, filtered_amp_spectrum.T, lowpass_filter)
        phase_spectrum_plot(signal_freqs, phase_spectrum.T, sampling_rate)

    else:
        amplitude__spectrum_map(outfile_path, filtered_amp_spectrum[:,:winlen], time_info, signal_freqs = signal_freqs, formato= ".png")

    # WHAT IS LEFT FROM THE FILTER SHOULD BE MOSTLY NOISE, RIGHT!?
    noise_spectrum = amp_spectrum - filtered_amp_spectrum
    noise_fourier = noise_spectrum * np.exp(1j*phase_spectrum)

    filtered_signal = np.fft.ifft(filtered_fourier).real
    noise_recostructed = np.fft.ifft(noise_fourier).real

    return filtered_signal, noise_recostructed



def amplitude__spectrum_map(outfile_path, amp_spectrum, time_info, signal_freqs, formato= ".png") -> None:
    outfile_name = os.path.basename(outfile_path)

    wave_num, wave_len = amp_spectrum.shape
    
    spectrum_length = round(wave_len/2)            # needed to make "manually" the real spectrum. I do not trust scipy, sorry
    signal_freqs = signal_freqs[:spectrum_length]
    amp_spectrum = amp_spectrum[:,:spectrum_length]

    time_ticks_waveforms = np.linspace(time_info[0], time_info[1], wave_num)

    plt.pcolormesh(time_ticks_waveforms ,signal_freqs,  amp_spectrum.T, cmap="gist_gray", norm="log")   
    plt.title('Amplitude Map of ' + outfile_name, fontsize = 12)
 
    plt.xlabel("Time")
    plt.ylabel("Frequency [$\mu $]")
    cbar= plt.colorbar(pad=0.04)
    cbar.set_label("Spectral Amplitude", fontsize = 14)

    plt.savefig(outfile_path + formato, dpi = 300)
    plt.close()
    

### Main code ###

In [39]:
#INPUT and OUTPUT FOLDERS: we must decide code-to-data relative path

machine_name = "Brava_2"
# xperiment_name = choice(("s0096","s0097","s0098"))
experiment_name = "s0098"
outdir_name = "images"
outdir_waveforms_plot = "waveforms"
outdir_amplitude_plot = "amplitude_maps"
outdir_spectrum_amp_plot = "spectrum_maps"

code_path = os.getcwd()
parent_folder = os.path.abspath(os.path.join(code_path, os.pardir))
indir_path = os.path.join(parent_folder,"experiments_"+ machine_name, experiment_name, "data_tsv_files")
outdir_path = indir_path.replace("data_tsv_files",outdir_name)
outdir_path_waveforms = os.path.join(outdir_path, outdir_waveforms_plot)
outdir_path_amplitude = os.path.join(outdir_path, outdir_amplitude_plot)
outdir_path_spectrum = os.path.join(outdir_path, outdir_spectrum_amp_plot)


###################### AGGIUSTARE: SPOSTARE DENTRO LE SINGOLE FUNZIONI ##################################
if not os.path.exists(outdir_path_spectrum):
    # os.makedirs(outdir_path_waveforms)
    # os.makedirs(outdir_path_amplitude)  
    os.makedirs(outdir_path_spectrum)  
########################################################################
    

### Some plot features ###
formato = '.png'
row_to_plot = 200  #which waveform to plot
remove_starting_noise = 0
highlight_start = 0
highlight_end = 0
ylim_plot = 500
xlim_plot= 20
ticks_steps_waveforms = 1 #[microseconds] plot ticks 
step_wf_to_plot = 10  #more waveforms on the same plot
amp_scale = 300  #color map for amplitude scale

### METADATA ###
encoding = 'iso8859' #just to read greek letters: there is a "mu" in the euroscan file            
    
for infile_name in tqdm(iterable=os.listdir(indir_path), desc='Opening files'):
    print(infile_name)
    ### IMPORT DATA ###
    file_path = os.path.join(indir_path, infile_name)
    # data = pd.read_csv(file_path, encoding = encoding, sep = '\t', skiprows = 4, header = None).dropna()
    with open(file_path,"rb") as infile:
        general = next(infile).decode(encoding)
        amplitude_scale = next(infile).decode(encoding)
        time_scale = next(infile).decode(encoding)
        #  print(time_scale)           # check what is going on below
        stupid_scale = next(infile).decode(encoding)
        # DEFINE TIME SCALE USING METADATA
        time_info = re.findall(r"\d+\.*\d*",time_scale)   # find all float in the string "time_scale" using regular expression:
                                                    # the first argument of findall must be a "regular expression". It start with an r so
                                                    # is an r-string, that means "\" can be read as a normal character
                                                    # The regular expression "\d+\.*\d*" means "match all the digits that are adiacent as
                                                    # a single entry and, if they are followed by a dot and some other digits, they are still
                                                    # the same numbers". So it gets as a single number both 1, 10, 1.4, 10.42 etc.
        time_info = [float(entry) for entry in time_info]           # just convert the needed info in float number
        time = np.arange(time_info[0],time_info[1],time_info[3])
        number_of_samples = time_info[2]
        time_ticks_waveforms = np.arange(time[0], time[-1], ticks_steps_waveforms)
        
        data_list = handling_empty_lines(infile, encoding, number_of_samples)
        
        data = np.array(data_list).astype(float)  #convert list in array
            
        
        #CHOOSE THE FILE PATH OF THE OUTPUT
        infile_name = infile_name.split('.')[0]    # remove extension
        outfile_path_waveforms = os.path.join(outdir_path_waveforms, infile_name)
        outfile_path_amplitude = os.path.join(outdir_path_amplitude, infile_name)
        outfile_path_spectrum = os.path.join(outdir_path_spectrum, infile_name)
        
        filtered_signal, noise_recostructed = signal2noise_separation_lowpass(outfile_path_spectrum, data, time_info, freq_cut=5)

            ### PLOT DATA ###
        try:
            # stacking_uw_plot(outfile_path_waveforms, data, step_wf_to_plot, time, remove_starting_noise, highlight_start, highlight_end, time_ticks_waveforms, ylim_plot, xlim_plot, formato)
            # amplitude_map(outfile_path_amplitude,data, amp_scale, time_info, formato)
            filtered_signal, noise_recostructed = signal2noise_separation_lowpass(outfile_path_spectrum, data, time_info, freq_cut=5)
            
        except:
            print(infile_name + "diocane")

        

Opening files:   0%|          | 0/7 [00:00<?, ?it/s]

PIS1_PIS2_ramp015_steel_2.bscan.tsv
PIS1_PIS2_ramp015_steel_7.bscan.tsv
PIS1_PIS2_ramp015_steel_hold_3.bscan.tsv
PIS1_PIS2_ramp015_steel_6.bscan.tsv
PIS1_PIS2_ramp015_steel_4.bscan.tsv
PIS1_PIS2_ramp015_steel.bscan.tsv
PIS1_PIS2_ramp015_steel_5.bscan.tsv


In [ ]:
# plt.plot(time[timecut:],single_waveform.T, color="gray")
# plt.plot(time[timecut:], filtered_signal.T, color="black")
# plt.legend()
# # plt.xlim([10,20])
# plt.show()

In [ ]:
# cut = round(0.1*len(time))

# plt.plot(time[:cut],single_waveform[:cut])
# plt.plot(time[:cut],filtered_signal[:cut]+25)
# plt.plot(time[:cut],noise_recostructed[:cut]+45)


### Nell exp 96 : massimo dello spettro e' a 500 KHz.
### Nell exp 97, il file a 2000 ns : il massimo dello spettro e' a 670 KHz.
### Nell exp 97 : il massimo dello spettro e' a 107 KHz.
### Il burst del noise iniziale appare chiaramente nello spettro di fase: da un trend discendente tra i 2-6 MHz e poi di nuovo tra i 7-11 MHz

Sbagliato io o ennesima cappellata?

In [ ]:
# # Is the noise gaussian?
# # Fit a normal distribution to the data:

# # mean and standard deviation
# data2hist = noise_recostructed
# mu, std = norm.fit(data2hist) 
# mu1, std1 = norm.fit(data2hist) 
 
# # Plot the histogram as an experimental PDF
# # plt.hist(data2hist, bins=100, density=True, alpha=0.6, color='b')
# counts, bins = np.histogram(noise_recostructed, bins=100)
# plt.hist(bins[:-1], bins, weights=counts, density=True, alpha=0.6, color="b")

# min_data = np.amin(data2hist)
# max_data= np.amax(data2hist)
# plt.xlim(min_data,max_data)

# # Plot the theoretical PDF
# xmin, xmax = plt.xlim()
# x = np.linspace(xmin, xmax, 100)
# p = norm.pdf(x, mu, std)
# p1 = norm.pdf(x, mu1, std1)
 
# plt.plot(x, p, 'k', linewidth=2)
# plt.plot(x, p1, 'r', linewidth=2)
# title = "Fit Values: {:.2f} and {:.2f}".format(mu, std)
# plt.title(title)
 
# plt.show()